In [1]:
import os, time, subprocess
from jinja2 import Template

variables = {}
with open("vars.txt") as myfile:
    for line in myfile:
        name, var = line.partition("=")[::2]
        variables[name.strip()] = str(var)

template_data = {"model_version": variables['model_version']}
applied_template = Template(open("inference-service.yaml").read())

rendered_template = applied_template.render(template_data)
print('Rendered Template: \r' + rendered_template)

with open("manifest.yaml", "w") as text_file:
    text_file.write(rendered_template)

!oc whoami
!oc apply -f manifest.yaml

print('Model deployment completed. Version: ' + variables['model_version'])

apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: versioned-wine-model
  namespace: wines
  labels:
    name: model-23.08.05-195259
    opendatahub.io/dashboard: 'true'
  annotations:
    openshift.io/display-name: model-23.08.05-195259
    serving.kserve.io/deploymentMode: ModelMesh
spec:
  predictor:
    model:
      modelFormat:
        name: sklearn
        version: '0'
      runtime: wine
      storage:
        key: aws-connection-wine-models
        path: model-23.08.05-195259
inferenceservice.serving.kserve.io/versioned-wine-model unchanged
Model deployment completed. Version: 23.08.05-195259
